# Imports and constants

In [ ]:
# import statemets

import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
import matplotlib.pyplot as plt
from customScripts import utilities as util
from customScripts import features as feat
from customScripts import onset

import librosa

In [ ]:
# parameters

#number of samples / second
sampling_rate = 100000 #96000

#length of frame in samples
frame_length = 2000 #2048

#number of samples used as offset for earch consecutive frame
hop_length = 1000 #1024

#number of mel frequency bins to use
bin_number = 40 #80

# size of the running window (in frames) for the threshold function
threshold_window_size = 60

#set the desired number of frames / second here
ground_thruth_conversion_const = 100 #2 # not used

# get train file paths
train_onsets_gt_paths, train_beats_gt_paths, train_bpm_gt_paths, train_onsets_audio_paths, train_beats_audio_paths, train_bpm_audio_paths = util.get_file_paths('music_data/train')
# get test file paths
test_onsets_gt_paths, test_beats_gt_paths, test_bpm_gt_paths, test_onsets_audio_paths, test_beats_audio_paths, test_bpm_audio_paths = util.get_file_paths('music_data/test')

# prediction_paths = ['predictions/Muppets-02-01-01.csv', 'predictions/Muppets-02-04-04.csv', 'predictions/Muppets-03-04-03.csv']
# file_lengths = [1547, 1548, 1539] #in seconds

# single file paths
single_audio_path = 'music_data/train/al_Media-103515(9.1-19.1).flac' #ah_development_guitar_2684_TexasMusicForge_Dandelion_pt1
single_gt_path_onsets = 'music_data/train/al_Media-103515(9.1-19.1).onsets'#ah_development_guitar_2684_TexasMusicForge_Dandelion_pt1
single_gt_path_beats = 'music_data/train/al_Media-103515(9.1-19.1).beats'
single_gt_path_bpm = 'music_data/train/al_Media-103515(9.1-19.1).bpm'

# File loading and feature extraction

In [ ]:
# load single audio
single_audio = util.load_audio(single_audio_path, sampling_rate)

In [ ]:
# compute single audio spectrogram
single_spectrogram = feat.compute_spectrogram(single_audio, frame_length, hop_length, bin_number)
single_spectrogram_T = single_spectrogram.transpose()

#print('frame number:',len(single_spectrogram_T))
#print('bin number:',len(single_spectrogram))

In [ ]:
# compute ground truth for single audio
single_gt_onsets = util.load_onsets_gt(single_gt_path_onsets, '/n')
single_gt_beats = util.load_beats_gt(single_gt_path_beats, '/n')
single_gt_bpm = util.load_bpm_gt(single_gt_path_bpm, '/n')

#print(single_gt_onsets)
#print(single_gt_beats)
#print(single_gt_bpm)

In [ ]:
# load onsets audio
train_onsets_audio = util.load_audios(train_onsets_audio_paths, sampling_rate)

In [ ]:
# compute onsets audio spectrogram

In [ ]:
# compute ground truth for onsets audio

In [ ]:
# load beats audio
train_beats_audio = util.load_audios(train_beats_audio_paths, sampling_rate)

In [ ]:
# compute beats audio spectrogram

In [ ]:
# compute ground truth for beats audio

In [ ]:
# load bpm audio
train_bpm_audio = util.load_audios(train_bpm_audio_paths, sampling_rate)

In [ ]:
# compute bpm audio spectrogram

In [ ]:
# compute ground truth for bpm audio
train_bpm_gt = util.load_ground_truths(train_bpm_gt_paths, '/n')
print(train_bpm_gt)

In [ ]:
#load ground truth, pad it with zeroes and adjust to frame number

#episode 1
"""
gt_1 = util.load_ground_truth(grount_truth_paths[0], '/n')
gt_1 = util.compute_0_padded_gt(gt_1, file_lengths[0])
gt_1 = util.adjust_gt_to_frames(gt_1, ground_thruth_conversion_const)
# append zeros so that lenght equals the frames list length
for i in range(len(spectrogram_1T) - len(gt_1)):
    gt_1.append(0)
print('ground truth size:',len(gt_1))
"""

In [ ]:
#concatenate training data
"""
gt_train = gt_1 + gt_2

spectrogram_train = np.concatenate((spectrogram_1T, spectrogram_2T), axis=0)

spectrogram_predict = spectrogram_3T

#spectrogram_train[3195] == spectrogram_2T[100]
"""
#for i,g in enumerate(gt_train):
#    if g == 1:
#        print(i)

# Classic onset detection method

In [ ]:
odf_single = onset.compute_odf(single_spectrogram)
peaks_single = onset.apply_threshold(odf_single, 61, True)

"""
for i, p in enumerate(peaks_single):
    if p > 0:
        print(i/100, '   ', p)
"""

maxima_frame_indices = onset.pick_local_peaks(peaks_single)
onset_time_stamps = maxima_frame_indices / 100

print(len(onset_time_stamps))
print(len(single_gt_onsets))
for i in range(61):
    if i < 60:
        print('predicted: ', onset_time_stamps[i], '   actual: ', single_gt_onsets[i])
    else:
        print('predicted: ', onset_time_stamps[i],)

# Classifier training and prediction for onsets

In [ ]:
# use a Multi-layer Perceptron classifier with default parameters
#clf = MLPClassifier()

In [ ]:
#clf.fit(spectrogram_train, gt_train)

In [ ]:
#prediction = clf.predict(spectrogram_predict)

In [ ]:
#np.savetxt(prediction_paths[2], prediction, delimiter="/n")

# Evaluation for onset detection

In [ ]:
"""
print('precision (micro average):',precision_score(gt_3, prediction, average='micro'))
print('precision (macro average):',precision_score(gt_3, prediction, average='macro'))
print('precision (weighted average):',precision_score(gt_3, prediction, average='weighted'))
"""

In [ ]:
"""
print('recall (micro average):',recall_score(gt_3, prediction, average='micro'))
print('recall (macro average):',recall_score(gt_3, prediction, average='macro'))
print('recall (weighted average):',recall_score(gt_3, prediction, average='weighted'))
"""

In [ ]:
"""
print('f1 (micro average):',f1_score(gt_3, prediction, average='micro'))
print('f1 (macro average):',f1_score(gt_3, prediction, average='macro'))
print('f1 (weighted average):',f1_score(gt_3, prediction, average='weighted'))
"""

In [ ]:
"""
# calculate false positive / true positive rate and area under curve
fpr, tpr, threshold = roc_curve(gt_3, prediction)
roc_auc = auc(fpr, tpr)

# plot ROC curve
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()
"""